In [1]:
from __future__ import absolute_import, division
from __future__ import print_function, unicode_literals
import pints.toy as toy
import pints
import numpy as np
import logging
import math
import matplotlib.ticker as mticker
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import sys
from numpy import inf
import copy 
import pickle
import time
import CMA as CMA

In [4]:
def CMA_on_single_var(opt, 
                 problem,
                 sampler,
                 sampler_x0, 
                 n_chains=1,
                 chain_size=100,
                 need_sensitivities=False, 
                eval_fun=['ESS']):
    
    optimizer_best_fxs = []
    optimizer_best_xs = []
    
    optimizer_best_fx = np.inf
    optimizer_best_x = 0
    
    for _ in range(100):
        
        # Getting the samples of hyper-parameters for the samplers
        optimizer_xs = opt.ask()
        
        # Saving the score of each sample of hyper-parameters
        optimizer_fxs = []
        
        # Evaluate performance for each hyper-parameter configuration
        for x in optimizer_xs:
            
            #our_x = [[x[0], x[1], x[2]], x[3]]
            our_x = [[x[0], x[1]], x[2]]
            
            # Initialise function evaluations and matrix for chains
            function_evaluations = 0 
            chains = []
            
            # Return array of samples for each chain
            for i in range(n_chains):

                # Create sampler object and set hyperparameter
                curr_x0 = sampler_x0[i]
                mcmc = sampler(curr_x0)
                mcmc.set_hyper_parameters(our_x)
                i_chain = []
                
                # Update until we have ``chain_size`` samples
                # Update function evaluations each time we use ask(),tell()
                while len(i_chain) < chain_size:
                    x = mcmc.ask()
                    if need_sensitivities:
                        fx, grad = problem.evaluateS1(x)
                        sample = mcmc.tell((fx, grad))
                    else:
                        fx, _ = problem.evaluateS1(x)
                        sample = mcmc.tell(fx)
                    function_evaluations += 1
                    if sample is not None:
                        i_chain.append(sample)
                
                # Append ith chain to list of chains       
                chains.append(i_chain)
                
            chains = np.array(chains, copy=True)
            optimizer_fx = 0
            # Calculate the score of the sampler with the given hyper-parameters
            # Get the KL if requested
            if 'KL' in eval_fun:
                kl = 0
                for chain in chains:
                    kl += problem.kl_divergence(chain)
                optimizer_fx = kl/len(chains)                
            
            elif 'KL-ITER' in eval_fun:
                kl = 0
                for chain in chains:
                    kl += problem.kl_divergence(chain)
                avg_kl = kl/len(chains)
                avg_iteration_count = function_evaluations / len(chains)
                optimizer_fx = avg_kl * avg_iteration_count
            
            # Get the ESS with Iteration if requested
            elif 'ESS-ITER' in eval_fun:
                ess = np.zeros(chains[0].shape[1])
                for chain in chains:
                    ess += np.array(pints._diagnostics.effective_sample_size(chain))
                ess /= len(chains)
                ess = np.min(ess)
                avg_iteration_count = function_evaluations / len(chains)
                optimizer_fx = math.sqrt(avg_iteration_count) / ess
            
            # Get the ESS if requested
            elif 'ESS' in eval_fun:
                ess = np.zeros(chains[0].shape[1])
                for chain in chains:
                    ess += np.array(pints._diagnostics.effective_sample_size(chain))
                ess /= len(chains)
                ess = np.min(ess)
                optimizer_fx = 1 / ess
            
            optimizer_fxs.append(optimizer_fx)
        opt.tell(optimizer_fxs)
        
        optimizer_best_fxs.append(opt.fbest())
        optimizer_best_xs.append(opt.xbest())
        
        if opt.fbest() < optimizer_best_fx:
            optimizer_best_fx = opt.fbest()
            optimizer_best_x = opt.xbest()
    
    print(optimizer_best_fx)
    print(optimizer_best_x)
    print()
    
    return optimizer_best_fxs, optimizer_best_xs

In [6]:
rosenbrock = pints.toy.RosenbrockLogPDF()
eggshape = pints.toy.SimpleEggBoxLogPDF(2,4)
Logistic = pints.toy.LogisticModel()

problem = eggshape

# Define the starting points of the samplers
sampler_rosenbrock_x0 = [
    [0,0],
    [3,8],
    [2,4],
]

sampler_egg_x0 = [
    [-10,-10],
    [0,0],
    [10,10],
]

sampler_x0 = sampler_egg_x0

# Define the starting point of the optimzer
dummy = pints.SliceDoublingMCMC([2, 4])
optimizer_x0 = [dummy.width()[0], dummy.width()[1], dummy.expansion_steps()]
boundary = pints.RectangularBoundaries([0,0,1],[20,20,50])

sampler = pints.SliceDoublingMCMC

for i in range(1):
    cma = CMA.CMAES(optimizer_x0, boundaries=boundary)
    fxs, xs =  CMA_on_single_var(cma, problem, sampler, 
                        sampler_x0, n_chains=3, chain_size=500, eval_fun=['ESS-ITER'])
    print("SCORES", fxs)
    print("PARAMS", xs)

1.4185921572602609
[3.55781155 4.59970624 1.87134159]

SCORES [3.4209690809498245, 3.4209690809498245, 3.4209690809498245, 3.4209690809498245, 3.037768384656112, 3.037768384656112, 3.037768384656112, 3.037768384656112, 3.037768384656112, 3.037768384656112, 3.037768384656112, 3.037768384656112, 3.037768384656112, 3.037768384656112, 3.037768384656112, 3.037768384656112, 3.037768384656112, 3.037768384656112, 3.037768384656112, 3.037768384656112, 3.037768384656112, 3.037768384656112, 3.037768384656112, 1.6366424919586287, 1.6366424919586287, 1.6366424919586287, 1.6366424919586287, 1.6366424919586287, 1.6366424919586287, 1.6366424919586287, 1.6366424919586287, 1.6366424919586287, 1.6366424919586287, 1.6366424919586287, 1.6366424919586287, 1.6366424919586287, 1.6366424919586287, 1.6366424919586287, 1.6366424919586287, 1.6366424919586287, 1.6366424919586287, 1.6366424919586287, 1.6366424919586287, 1.6366424919586287, 1.4185921572602609, 1.4185921572602609, 1.4185921572602609, 1.41859215726026

In [2]:
dummy = pints.SliceDoublingMCMC([2, 4])
optimizer_x0 = [dummy.width()[0], dummy.width()[1], dummy.expansion_steps()]
print(optimizer_x0)

[0.2, 0.4, 10]


## Results for the Rosenbrock Problem


In [ ]:
2.65165856770783
[7.66063406 7.94001634 6.32656355]

2.547566318306533
[ 4.16839149  5.68204453 10.61587009]

2.610463537959919
[ 3.05592567  4.41440798 12.79150919]

2.6958410930908703
[ 8.473456    3.58357067 11.72819232]

2.81859260274518
[ 1.53479358  1.62903019 15.31999008]

## Results for the Egg Shape Problem

In [ ]:
1.4096887136687746
[5.26178205 5.45247427 1.92230768]

1.3147810455223032
[5.94037614 7.62648458 1.75172792]

1.3489126362505708
[6.14576926 5.59790442 1.57472481]

1.4114654502722221
[5.1707506  5.01498032 1.86376232]

1.4028370578703422
[4.87842409 6.21041138 1.11259927]

## Logistic Problem

In [ ]:
import pints
import pints.toy as toy
import pints.plot
import numpy as np
import matplotlib.pyplot as plt

class ExampleModel(pints.ForwardModel):
    
    def __init__(self):
        self.model = toy.LogisticModel()
    def simulate(self, x, times):
        return self.model.simulate([x[0]/1000, x[1]], times)
    def simulateS1(self, x, times):
        values, gradient = self.model.simulateS1([x[0]/1000, x[1]], times)
        return values, gradient
    def n_parameters(self):
        return 2

# Then create an instance of our new model class
model = ExampleModel()

In [ ]:
# Create some toy data
real_parameters = [0.015*10000, 500]
times = np.linspace(0, 1000, 1000)
org_values = model.simulate(real_parameters, times)

# Add noise
noise = 10
values = org_values + np.random.normal(0, noise, org_values.shape)
real_parameters = np.array(real_parameters + [noise])

# Get properties of the noise sample
noise_sample_mean = np.mean(values - org_values)
noise_sample_std = np.std(values - org_values)

# Create an object with links to the model and time series
problem = pints.SingleOutputProblem(model, times, values)

# Create a log-likelihood function (adds an extra parameter!)
log_likelihood = pints.GaussianLogLikelihood(problem)

# Create a uniform prior over both the parameters and the new noise variable
log_prior = pints.UniformLogPrior(
    [0.01*10000, 400, noise * 0.1],
    [0.02*10000, 600, noise * 100],
)

# Create a posterior log-likelihood (log(likelihood * prior))
log_posterior = pints.LogPosterior(log_likelihood, log_prior)

# Choose starting points for 3 mcmc chains
num_chains = 3
sampler_logistic_x0 = [
    real_parameters * 1.01,
    real_parameters * 0.99,
    real_parameters * 1.01
]

In [ ]:
problem = log_posterior

sampler_x0 = sampler_logistic_x0

# Define the starting point of the optimzer
dummy = pints.SliceDoublingMCMC([2, 4, 10])
optimizer_x0 = [dummy.width()[0], dummy.width()[1], dummy.width()[2], dummy.expansion_steps()]


boundary = pints.RectangularBoundaries([0,0,0,1],[20,20,20,15])

sampler = pints.SliceDoublingMCMC

for i in range(5):
    cma = CMA.CMAES(optimizer_x0, boundaries=boundary)
    fxs, xs =  CMA_on_single_var(cma, problem, sampler, 
                        sampler_x0, n_chains=3, chain_size=100, eval_fun=['ESS'])

In [3]:

dummy = pints.SliceDoublingMCMC([2, 4, 10])
optimizer_x0 = [dummy.width()[0], dummy.width()[1], dummy.width()[2], dummy.expansion_steps()]
print(optimizer_x0)

[0.2, 0.4, 1.0, 10]


In [ ]:
1.6217118439833906
[3.17171131 6.3090571  4.28788366 4.73189072]

1.6093355480002016
[18.84696198  5.68989611  5.32926045 10.56924945]

1.6182910721347594
[6.86305237 7.03818512 1.36405241 8.93229663]

1.6131251090905396
[10.83416564  3.64714311  4.65312227  6.31316515]

1.618273135380834
[10.70865917 15.1015873   2.57134735 14.77128816]